对象接口协议

- 只要实现了类的某些指定方法，就可以称为实现了某个接口,就获得相应的相关能力
  - 比如:容器,求长度,迭代,切片,索引,上下文管理器


### 容器

- 有 in 方法的能力,判断前面的对象是否在后面的对象中
  - 'a' in ['a','b','c'] # True
  - 'a' not in ['a','b','c'] # False
  - contains,return 真假值


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
class myCls:
    def __init__(self, data) -> None:
        self.items = data

    def __contains__(self, k):
        for _, j in self.items:
            if j == k:
                return True
        return False

    # return none
    # pass默认返回false


data = myCls([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)])
5 in data, 1 in data

(True, False)

In [7]:
# 切片操作
class Hero:
    def __init__(self, hmc, tag) -> None:
        self.hmc = hmc

    def __getitem__(self, index):  # index是切片对象
        hmclist = self.hmc.split()
        return hmclist[index]


ls = Hero('武松 李逵 宋江 鲁智深', '梁山')
ls[0], ls[:2], ls[1:2]

('武松', ['武松', '李逵'], ['李逵'])

In [4]:
from typing import Any


class Hero:
    def __init__(self, hmc, tag) -> None:
        self.hmc = hmc

    def __getitem__(self, index):  # index是切片对象
        hmclist = self.hmc.split()
        if isinstance(index, slice):
            return hmclist[index.start:index.stop:index.step]
        else:
            return hmclist[index]

    def __getattr__(self, attr__name: str) -> Any:
        hmclist = self.hmc.split()
        if attr__name in hmclist:
            return '{}，第{}条好汉'.format(attr__name, hmclist.index(attr__name)+1)
        else:
            return None


ls = Hero('武松 李逵 宋江 鲁智深', '梁山')
ls[0], ls[:2], ls.武松, ls.李逵, ls.宋江, ls.鲁智深, ls.林冲

('武松', ['武松', '李逵'], '武松，第1条好汉', '李逵，第2条好汉', '宋江，第3条好汉', '鲁智深，第4条好汉', None)

In [5]:
# 对象化字典
from typing import Any


class objDict(dict):
    def __getattr__(self, attr__name: str) -> Any:
        c = self[attr__name]
        if isinstance(c, dict):
            return objDict(c)  # 递归调用，将字典转换为对象，然后返回
        return c


d = dict(a=1, b=2, c=3)
d = dict(a=1, b=2, c=3, d={'aa': 1, 'bb': 2, 'cc': 3})
d['a']
d = objDict(d)
d.a, d.b, d.c, d.d
d.d.aa  # 嵌套字典

1

(1, 2, 3, {'aa': 1, 'bb': 2, 'cc': 3})

1

### 上下文管理器

将某个类的对象使用范围，进入或者退出该范围时候，会自动触发特殊的操作，比如关闭连接，释放内存等等
**enter**和**exit**方法

- **enter**方法
  - with 语句块开始执行时，会调用该方法，获取该方法的返回值，赋值给 as 后面的变量
- **exit**方法

  - with 语句块执行完毕后，会调用该方法，释放资源

- with 语句
  - with open('a.txt') as f:
    f.read()
  - with 语句会自动调用 open 函数返回的对象的**enter**方法，获取该方法的返回值，赋值给 as 后面的变量
  - with 语句块执行完毕后，会自动调用 open 函数返回的对象的**exit**方法，释放资源


In [6]:
import sqlite3


class mySqlites3:
    def __init__(self, name) -> None: # 初始化，先执行
        self.dbname = name

    def __enter__(self) -> sqlite3.Connection: # 进入with语句时执行
        self.conn = sqlite3.connect(self.dbname)
        return self.conn  # 返回连接对象，必须要返回

    def __exit__(self, exc_type, exc_val, exc_tb) -> None:  # 这里必须要有4个参数才能正常运行
        self.conn.close()
        return True


db = '../data/xwlb.db'
with mySqlites3(db) as conn:
    c = conn.execute("""select *from lb00 """)  # 返回一个游标对象，作用是遍历查询结果
    data = [x[1] for x in c]

data[:10]

['习近平春节前夕赴江西看望慰问广大干部群众 祝全国各族人民健康快乐吉祥 祝改革发展人民生活蒸蒸日上',
 '李克强在宁夏考察时强调 以新理念新动能推结构升级 促进经济发展和社会公正',
 '新华社述评：正风肃纪 把纪律挺在前面',
 '国家发改委：中国经济结构优化 新动力正在积聚',
 '深化改革带来更多获得感',
 '国内联播快讯',
 '外交部：希望朝方保持克制 慎重行事',
 '叙利亚问题日内瓦和谈陷入僵局',
 '传承红色基因 让人民生活蒸蒸日上',
 '李克强主持召开国务院常务会议']